# Distributed Training Strategies

- Synchronous (Different parts of the dataset during the same time, at the end the gradients are aggregated and used to update one model)
- Asynchronous (All workers are training at the same time and updating weights asynchronously)

Training a tensorflow model on multiple GPUs, computers etc

Tensorflow implement distributed Strategies in two ways
- Mirrored Strategy
- Multi Worker Mirrored Strategy

## Build the Complete Code

In [1]:
import time
import numpy as np
import tensorflow as tf

In [2]:
# Loading and preprocessing the dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0

In [3]:
# Reshaping the data
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

print(f"Reshaped training features - {X_train.shape}")

Reshaped training features - (60000, 784)


## Distributed Training - Defining Normal Non-Distributed Model

In [4]:
model_normal = tf.keras.models.Sequential()
model_normal.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model_normal.add(tf.keras.layers.Dropout(rate=0.2))
model_normal.add(tf.keras.layers.Dense(units=10, activation='softmax'))
model_normal.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

## Defining the Distributed Strategy

In [5]:
distribute = tf.distribute.MirroredStrategy()
with distribute.scope():
  model_distributed = tf.keras.models.Sequential()
  model_distributed.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
  model_distributed.add(tf.keras.layers.Dropout(rate=0.2))
  model_distributed.add(tf.keras.layers.Dense(units=10, activation='softmax'))
  model_distributed.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


## Speed Comparison between normal training and distributed training process

In [6]:
start_time = time.time()
model_normal.fit(X_train, y_train, epochs=10, batch_size=25)
print("Normal training took: {}".format(time.time() - start_time))

Epoch 1/10
2400/2400 [==============================] - 6s 2ms/step - loss: 0.2824 - sparse_categorical_accuracy: 0.9170
Epoch 2/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.1374 - sparse_categorical_accuracy: 0.9587
Epoch 3/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.1010 - sparse_categorical_accuracy: 0.9694
Epoch 4/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.0855 - sparse_categorical_accuracy: 0.9730
Epoch 5/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.0716 - sparse_categorical_accuracy: 0.9775
Epoch 6/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.0640 - sparse_categorical_accuracy: 0.9794
Epoch 7/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.0581 - sparse_categorical_accuracy: 0.9806
Epoch 8/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.0502 - sparse_categorical_accuracy: 0.9839
Epoch 9/10
2400/2400 [==========

In [7]:
start_time = time.time()
model_distributed.fit(X_train, y_train, epochs=10, batch_size=25)
print("Distributed training took: {}".format(time.time() - start_time))

Epoch 1/10
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
2400/2400 [==============================] - 8s 3ms/step - loss: 0.2855 - sparse_categorical_accuracy: 0.9172
Epoch 2/10
2400/2400 [=================

We see from above that distributed training took longer than normal training because in this case in distributed training the load was shared between GPU and CPU of the computer. Since CPU is slower than GPU , hence distributed training took longer.

In cases like this where a load is distributed between GPU and CPU . it will perform slower than the entire training happening on GPU ... hence the result above